# The JuMP ecosystem for mathematical optimization: MOI

This notebook is part of a workshop at [JuliaCon 2018](http://juliacon.org/2018/) and is based on materials and notebooks from various sources including the [JuliaOpt notebooks](https://github.com/JuliaOpt/juliaopt-notebooks), the [2018 ISCO Spring School](https://github.com/joehuchette/ISCO-spring-school) and the [second annual JuMP-dev workshop](http://www.juliaopt.org/meetings/bordeaux2018/).

## Disclaimer
This notebook is only working under the versions:

- JuMP 0.19 (unreleased, but currently in master)

- MathOptInterface 0.4.1

- GLPK 0.6.0

In [1]:
using JuMP  
using MathOptInterface # Replaces MathProgBase
# shortcuts
const MOI = MathOptInterface
const MOIU = MathOptInterface.Utilities

using GLPK # Loading the GLPK module for using its solver

## MOI and JuMP

In [2]:
model = Model()
@variable(model, x[1:2])
@constraint(model, linear1, 2*x[1] +   x[2] <= 1)  # save reference linear1 for this constraint
@constraint(model, 2*x[1] + 2*x[2] <= 1) 
@constraint(model, x[1]^2 + x[2]^2 <= 1)  
@constraint(model, [1;x] in SecondOrderCone())  # ||x|| <= 1
@objective(model, Max, x[1] + x[2]) 

Behind every JuMP model there is a MOI model (used to be a MPB model)

In [3]:
typeof(model.moibackend)

MathOptInterface.Bridges.LazyBridgeOptimizer{MathOptInterface.Utilities.CachingOptimizer{MathOptInterface.AbstractOptimizer,MathOptInterface.Utilities.UniversalFallback{JuMP.JuMPMOIModel{Float64}}},MathOptInterface.Bridges.AllBridgedConstraints{Float64}}

### Recovering variables:

In [4]:
xx = MOI.get(model.moibackend,MOI.ListOfVariableIndices()) 

2-element Array{MathOptInterface.VariableIndex,1}:
 MathOptInterface.VariableIndex(1)
 MathOptInterface.VariableIndex(2)

In [5]:
x[1].index == xx[1]

true

In [6]:
x[1].m == model

true

In [7]:
typeof(x[1])

JuMP.VariableRef

JuMP variable : MOI variable + owning model
```julia
struct VariableRef 
    m::Model
    index::MOI.VariableIndex
end
```

MOI variable : Type-safe unique and not necessarily consecutive index
```julia
struct VariableIndex
    value::Int64
end
```

### Recovering constraints:

In [8]:
L,Q,C = MOI.get(model.moibackend,MOI.ListOfConstraints())

3-element Array{Tuple{DataType,DataType},1}:
 (MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64})   
 (MathOptInterface.ScalarQuadraticFunction{Float64}, MathOptInterface.LessThan{Float64})
 (MathOptInterface.VectorAffineFunction{Float64}, MathOptInterface.SecondOrderCone)     

MOI constraints are of the form 
$$
f(x) \in S
$$
were
* $f$ is an ``MOI.AbstractFunction`` (e.g. ``MOI.ScalarAffineFunction`` = $a^T x +b$ for some $a,b$) and
* $S$ is an ``MOI.AbstractSet`` (e.g. ``MOI.LessThan`` = $\{y\,:\, y\leq u\}$ for some $u$)

#### Linear inequalities (``MOI.ScalarAffineFunction``-in-``MOI.LessThan``):

In [9]:
linear = MOI.get(model.moibackend,MOI.ListOfConstraintIndices{MOI.ScalarAffineFunction{Float64}, MOI.LessThan{Float64}}())  #MOI.get(model.moibackend,MOI.ListOfConstraintIndices{L...}())

2-element Array{MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},1}:
 MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}}(1)
 MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}}(2)

In [10]:
linear1.index == linear[1]

true

In [11]:
fieldnames(linear1)

2-element Array{Symbol,1}:
 :m    
 :index

In [12]:
linear1.m == model

true

In [13]:
typeof(linear1)

JuMP.ConstraintRef{JuMP.Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}}}

JuMP constraint : MOI constraint + owning model
```julia
struct ConstraintRef{C}
    m::Model
    index::C             # C = ConstraintIndex{F,S} for some F,S
end
```

MOI constraint : Type-safe wrapper of index to constraint of class F-in-S. Index not necessarily consecutive and unique within constraint class
```julia
struct ConstraintIndex{F,S}
    value::Int64
end
```

Linear inequalities 1 : 
* ``2*x[1] + x[2] <= 1``, or
* $a^T x + b \in \{y:y\leq u\}$ for $a=(2,1)^T$, $b=0$ and $u=0$ 

$a=(2,1)^T$:

In [14]:
MOI.get(model.moibackend,MOI.ConstraintFunction(),linear[1]).terms

2-element Array{MathOptInterface.ScalarAffineTerm{Float64},1}:
 MathOptInterface.ScalarAffineTerm{Float64}(2.0, MathOptInterface.VariableIndex(1))
 MathOptInterface.ScalarAffineTerm{Float64}(1.0, MathOptInterface.VariableIndex(2))

$b=0$:

In [15]:
MOI.get(model.moibackend,MOI.ConstraintFunction(),linear[1]).constant

0.0

$u=0$:

In [16]:
MOI.get(model.moibackend,MOI.ConstraintSet(),linear[1]).upper

1.0

Similar for othet constrsints:

In [17]:
MOI.get(model.moibackend,MOI.ListOfConstraintIndices{MOI.ScalarQuadraticFunction{Float64}, MOI.LessThan{Float64}}())  #MOI.get(model.moibackend,MOI.ListOfConstraintIndices{Q...}())

1-element Array{MathOptInterface.ConstraintIndex{MathOptInterface.ScalarQuadraticFunction{Float64},MathOptInterface.LessThan{Float64}},1}:
 MathOptInterface.ConstraintIndex{MathOptInterface.ScalarQuadraticFunction{Float64},MathOptInterface.LessThan{Float64}}(3)

In [18]:
MOI.get(model.moibackend,MOI.ListOfConstraintIndices{MOI.VectorAffineFunction{Float64}, MOI.SecondOrderCone}())  #MOI.get(model.moibackend,MOI.ListOfConstraintIndices{C...}())

1-element Array{MathOptInterface.ConstraintIndex{MathOptInterface.VectorAffineFunction{Float64},MathOptInterface.SecondOrderCone},1}:
 MathOptInterface.ConstraintIndex{MathOptInterface.VectorAffineFunction{Float64},MathOptInterface.SecondOrderCone}(4)

## MOI Constraints

| Mathematical Constraint       | MOI Function                 | MOI Set        |
|-------------------------------|------------------------------|----------------|
| $a^Tx = b$                  | `ScalarAffineFunction`       | `EqualTo`      |
| $l \le a^Tx \le u$          | `ScalarAffineFunction`       | `Interval`     |
| $x_i \le u$                 | `SingleVariable`             | `LessThan`     |
| $Ax + b \in \mathbb{R}_+^n$ | `VectorAffineFunction`       | `Nonnegatives` |
| $\lVert Ax + b\rVert_2 \le c^Tx + d$                        | `VectorAffineFunction`       | `SecondOrderCone` |
| $y \exp (x/y) \le z, y > 0$  | `VectorOfVariables`       | `ExponentialCone`                  |
| $x \in \mathbb{R}^{d\left(d+1\right)/2}$,◹$(x)\in \text{PSD} \subseteq \mathbb{R}^{d\times d}$                                       | `VectorOfVariables`          | `PositiveSemidefiniteConeTriangle` |
| $x^TQx + a^Tx + b \ge 0$    | `ScalarQuadraticFunction`    | `GreaterThan`                 |
| $x_i \in \mathbb{Z}$                                                                     | `SingleVariable`    | `Integer`                          |
| $x_i \in \{0,1\}$                                                                        | `SingleVariable`    | `ZeroOne`                          |
| $x_i \in \{0\} \cup [l,u]$                                                               | `SingleVariable`    | `Semicontinuous`                   |
| At most one component of $x$ can be nonzero                                              | `VectorOfVariables` | `SOS1`                             |

MOI includes standard functions and sets, and allows for some extensions. See [MOI manual](http://www.juliaopt.org/MathOptInterface.jl/stable/index.html) for more details on [supported sets and functions](http://www.juliaopt.org/MathOptInterface.jl/stable/apimanual.html#Standard-form-problem-1) and more [constraint examples](http://www.juliaopt.org/MathOptInterface.jl/stable/apimanual.html#Constraints-by-function-set-pairs-1).

### Note: there are multiple ways to write the same constraint. For more details and discussions see [Constraint Bridges](#Constraint-Bridges).

## A Complete MOI Model

In [19]:
using MathOptInterface
const MOI = MathOptInterface
using GLPK

# Solve the binary-constrained knapsack problem: max c'x: w'x <= C, x binary using GLPK.

c = [1.0, 2.0, 3.0]
w = [0.3, 0.5, 1.0]
C = 3.2

numvariables = length(c)

optimizer = GLPK.GLPKOptimizerMIP()

# create the variables in the problem
x = MOI.addvariables!(optimizer, numvariables)

# set the objective function
objective_function = MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(c, x), 0.0)
MOI.set!(optimizer, MOI.ObjectiveFunction{MOI.ScalarAffineFunction{Float64}}(), objective_function)
MOI.set!(optimizer, MOI.ObjectiveSense(), MOI.MaxSense)

# add the knapsack constraint
knapsack_function = MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(w, x), 0.0)
MOI.addconstraint!(optimizer, knapsack_function, MOI.LessThan(C))

# add integrality constraints
for i in 1:numvariables
    MOI.addconstraint!(optimizer, MOI.SingleVariable(x[i]), MOI.ZeroOne())
end

# all set
MOI.optimize!(optimizer)

termination_status = MOI.get(optimizer, MOI.TerminationStatus())
objvalue = MOI.canget(optimizer, MOI.ObjectiveValue()) ? MOI.get(optimizer, MOI.ObjectiveValue()) : NaN
if termination_status != MOI.Success
    error("Solver terminated with status $termination_status")
end

@assert MOI.get(optimizer, MOI.ResultCount()) > 0

result_status = MOI.get(optimizer, MOI.PrimalStatus())
if result_status != MOI.FeasiblePoint
    error("Solver ran successfully did not return a feasible point. The problem may be infeasible.")
end
primal_variable_result = MOI.get(optimizer, MOI.VariablePrimal(), x)

@show objvalue
@show primal_variable_result

objvalue = 6.0
primal_variable_result = 

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

[1.0, 1.0, 1.0]


## Model modifications

In [20]:
model = Model() 
@variable(model, x[1:2])
@constraint(model, linear1, 2 * x[1] +   x[2] <= 1)  # save reference linear1 for this constraint
@constraint(model, linear2, 2 * x[1] + 2 * x[2] <= 1) 
@constraint(model, quadratic, x[1]^2 + x[2]^2 <= 1)  
@constraint(model, socp, [1;x] in SecondOrderCone())  # ||x|| <= 1
@objective(model, Max, x[1] + x[2]) 

delete variables or constraints

In [21]:
if MOI.candelete(model.moibackend,x[1].index)
    MOI.delete!(model.moibackend,x[1].index)
end
if MOI.candelete(model.moibackend,linear1.index)
    MOI.delete!(model.moibackend,linear1.index)
end

modify constraints

In [22]:
if MOI.canmodify(model.moibackend, typeof(linear2.index), MOI.ScalarCoefficientChange{Float64})
    MOI.modify!(model.moibackend, linear2.index, MOI.ScalarCoefficientChange(x[1].index, 3.0))
end

or completely transform them 

In [23]:
rsocp = MOI.transform!(model.moibackend, socp.index, MOI.RotatedSecondOrderCone(3))

MathOptInterface.ConstraintIndex{MathOptInterface.VectorAffineFunction{Float64},MathOptInterface.RotatedSecondOrderCone}(5)

In [24]:
rsocp != socp

true

In [25]:
MOI.isvalid(model.moibackend,socp.index)

false

### Note: Modifications are as efficient "as possible" 

## MOI Attributes

### For MOI Models

In [26]:
model = GLPK.GLPKOptimizerMIP()
var_index = MOI.addvariable!(model)
constr_index = MOI.addconstraint!(model,MOI.SingleVariable(var_index),MOI.LessThan(10.0))

MathOptInterface.ConstraintIndex{MathOptInterface.SingleVariable,MathOptInterface.LessThan{Float64}}(1)

for building models...

In [27]:
MOI.set!(model,MOI.ObjectiveSense(),MOI.MaxSense)
MOI.set!(model, MOI.ObjectiveFunction{MOI.SingleVariable}(), MOI.SingleVariable(var_index))

0

and more...

In [28]:
if MOI.canset(model,MOI.VariablePrimalStart(),typeof(var_index))
    MOI.set!(model,MOI.VariablePrimalStart(),var_index,5.0)
end
if MOI.canset(model,MOI.ConstraintDualStart(),typeof(constr_index))
    MOI.set!(model,MOI.ConstraintDualStart(),constr_index,5.0)
end

### Also for JuMP models

In [29]:
model = Model()

A JuMP Model

some attributes have custom macro syntax

In [30]:
@variable(model, x >= 10.0, start = 5.0)
@objective(model, Max, x)

MathOptInterface.SingleVariable(MathOptInterface.VariableIndex(1))

others use MOI functions

In [31]:
bound_ref = JuMP.LowerBoundRef(x).index
if MOI.canset(model.moibackend,MOI.ConstraintDualStart(),typeof(bound_ref))
    MOI.set!(model.moibackend,MOI.ConstraintDualStart(),bound_ref,10.0)
end

10.0

## JuMP and MOI

Note: Need the following patch [implemented in MOI master](https://github.com/JuliaOpt/MathOptInterface.jl/pull/409), but not on release v0.4.1

In [32]:
MOI.canget(m::MOIU.CachingOptimizer, IdxT::Type{<:MOI.Index}, name::String) = MOI.canget(m.model_cache, IdxT, name)
MOI.get(m::MOIU.CachingOptimizer, IdxT::Type{<:MOI.Index}, name::String) = MOI.get(m.model_cache, IdxT, name)

In [34]:
model = Model(with_optimizer(GLPK.GLPKOptimizerLP))
@variable(model, x >= 0)
@variable(model, y >= 0)
@constraint(model, inequality, x + y <= 1)
@objective(model, Max, x + y)

In [12]:
#@macroexpand @variable(model, x >= 0)

`@variable(model, x >= 0)`:

1. Create JuMP/MOI variable and set its attributes:
```julia
x = JuMP.addvariable(model,[...],"x")
```
2. Save JuMP variable in symbol-keyed dictionary:
```julia
JuMP.registervar(model, :x, x)
```

In [22]:
#@macroexpand @constraint(model, inequality, x + y <= 1)

`@constraint(model, inequality, x + y <= 1)`:

1. Create JuMP/MOI constraint (function and set):
```julia
inequality = JuMP.addconstraint(model,[...],"inequality")
``` 
2. Save JuMP constraint in symbol-keyed dictionary:
```julia
JuMP.registercon(model, :inequality, inequality)
``` 

#### JuMP stores data only on MOI

```julia
function addconstraint(m::Model, c::AbstractConstraint, name::String="")
    f, s = moi_function_and_set(c)
    [...]
    cindex = MOI.addconstraint!(m.moibackend, f, s)::ConstraintIndex{F,S} 
    cref = ConstraintRef(m, cindex)::ConstraintRef{ConstraintIndex{F,S}}
    if !isempty(name)
        MOI.set!(m.moibackend, MOI.ConstraintName(), cindex,name)
    end
    return cref
end
```
JuMP constraint:
```julia
struct ConstraintRef{C}
    m::Model
    index::C             # C = ConstraintIndex{F,S} for some F,S
end
```

We can recover MOI constraints and variables by their "name":

In [35]:
MOI.get(model.moibackend, MOI.ConstraintIndex, "inequality")  == inequality.index

true

In [36]:
MOI.get(model.moibackend, MOI.VariableIndex, "x")  == x.index

true

and get the name of a constraint or variable:

In [37]:
MOI.get(model.moibackend, MOI.ConstraintName(),inequality.index)

"inequality"

In [38]:
MOI.get(model.moibackend, MOI.VariableName(),x.index)

"x"

Note: Simpler syntax to get name of JuMP variable will be available by [v0.19 release](https://github.com/JuliaOpt/JuMP.jl/issues/1184) 

#### JuMP does store dictionary to recover variables/constraints by symbol

In [39]:
model[:x] == x

true

In [40]:
model[:inequality] == inequality

true

usefull when working with multiple models with same names:

In [41]:
model1 = Model(with_optimizer(GLPK.GLPKOptimizerLP))
@variable(model1, x >= 0)
@variable(model1, y >= 0)
@constraint(model1, inequality, x + y <= 1)
@objective(model1, Max, x + y)

model2 = Model(with_optimizer(GLPK.GLPKOptimizerLP))
@variable(model2, x >= 0)
@variable(model2, y >= 0)
@constraint(model2, inequality, 2x + y <= 1)
@objective(model2, Max, x + y)

`inequality` for `model1` was overwritten by `inequality` for `model2`: 

In [42]:
inequality

inequality : 2 x + y ≤ 1.0

we can recover `inequality` for `model1` from `model1[...]`:

In [43]:
model1[:inequality]

inequality : x + y ≤ 1.0

### A recap of the four types of variables/constraints in JuMP

Julia variables v/s mathematical optimization variables?! 

In [44]:
model = Model(with_optimizer(GLPK.GLPKOptimizerLP))
@variable(model, x[1:3] >= 0, basename="Xvar")
@variable(model, y >= 0)
@constraint(model, inequality, sum(x) + y <= 1)
@objective(model, Max, sum(x) + y)

MOI variable: a unique index within a model

In [45]:
y.index

MathOptInterface.VariableIndex(4)

In [46]:
x[2].index

MathOptInterface.VariableIndex(2)

MOI variable name: a unique string associated to a MOI variable.

In [47]:
MOI.get(model.moibackend, MOI.VariableName(),y.index)

"y"

In [48]:
MOI.get(model.moibackend, MOI.VariableName(),x[2].index)

"Xvar[2]"

In [49]:
MOI.set!(model.moibackend, MOI.VariableName(),x[3].index,"lastXvar")

MathOptInterface.VariableIndex(3)

In [50]:
x

3-element Array{JuMP.VariableRef,1}:
 Xvar[1] 
 Xvar[2] 
 lastXvar

Single JuMP variable: Reference to a MOI variable in a model. A Julia variable of type `JuMP.VariableRef`

In [51]:
typeof(y)

JuMP.VariableRef

In [52]:
y.index

MathOptInterface.VariableIndex(4)

In [53]:
y.m == model

true

Container of JuMP variables: well... a container of `JuMP.VariableRef` 

In [54]:
typeof(x)

Array{JuMP.VariableRef,1}

### Interesting Examples:

In [55]:
model = Model(with_optimizer(GLPK.GLPKOptimizerLP))
x = @variable(model, [1:3])

3-element Array{JuMP.VariableRef,1}:
 noname
 noname
 noname

In [56]:
x

3-element Array{JuMP.VariableRef,1}:
 noname
 noname
 noname

In [58]:
model = Model(with_optimizer(GLPK.GLPKOptimizerLP))
@variable(model, [1:3], basename=string(rand()))

3-element Array{JuMP.VariableRef,1}:
 0.7458337863247182[1]
 0.8751360655391893[2]
 0.3736794369071459[3]

In [60]:
using DataStructures
myvariables = Stack(JuMP.VariableRef)
model = Model(with_optimizer(GLPK.GLPKOptimizerLP))
push!(myvariables,@variable(model, y >= 0))
push!.(myvariables,@variable(model, z[1:3] >= 0))
pop!(myvariables)

z[3]

In [61]:
@show myvariables

DataStructures.Stack{JuMP.VariableRef}(Deque [JuMP.VariableRef[y, z[1], z[2]]])

myvariables = DataStructures.Stack{JuMP.VariableRef}(Deque [JuMP.VariableRef[y, z[1], z[2]]])


## JuMP Solver Modes 

In [63]:
model = Model(with_optimizer(GLPK.GLPKOptimizerLP))
@variable(model, x >= 0)
@variable(model, y >= 0)
@constraint(model, inequality, x + y <= 1)
@objective(model, Max, x + y)

JuMP has 3 solver modes:

In [64]:
JuMP.ModelMode

Enum JuMP.ModelMode:
Automatic = 0
Manual = 1
Direct = 2

Default mode is `Automatic`

In [65]:
JuMP.mode(model) == JuMP.Automatic

true

In [3]:
c = [1; 3; 5; 2] 

A= [
     1 1 9 5;
     3 5 0 8;
     2 0 6 13
    ]

b = [7; 3; 5] 

m, n = size(A); # m = number of rows of A, n = number of columns of A

### One Extreme: Automatic Mode = "Almost" Old JuMP

Solver is a `CachingOptimizer` which keeps a cache of the model data (remember all data is stored in MOI) that can be incrementally modified in every possible way. If the "true" internal model/solver (e.g. `GLPK.GLPKOptimizerLP()`'s model below) does not support an incremental modification, the `CachingOptimizer` discards the internal model and restores the model from the cache after the modification (i.e. it re-builds the complete model).
* Pros: No need to worry about what modifications are valid.
* Cons: No information about efficiency of modifications (may be re-creating the whole model for each modification)

**Note:** Automatic mode also provides automatic constraint transformations. See [Constraint Bridges](#Constraint-Bridges)

In [90]:
model = JuMP.Model(with_optimizer(GLPK.GLPKOptimizerLP, msg_lev = 4))

@variable(model, x[1:n] >= 0) 
@constraint(model, [i=1:m], sum(A[i,j]*x[j] for j in 1:n) == b[i])
@objective(model, Min, sum(c[j]*x[j] for j in 1:n)) 

#SOLVE IT
#--------

# finally optimize the model
@time begin
JuMP.optimize(model) # solves the model
end

# TEST SOLVER STATUSES and DISPLAY THE RESULTS
#---------------------------------------------
@show JuMP.hasresultvalues(model)
@show JuMP.terminationstatus(model) == MOI.Success
@show JuMP.primalstatus(model) == MOI.FeasiblePoint

println("Objective value: ", JuMP.objectivevalue(model))
println("Optimal solution is x = \n", JuMP.resultvalue.(x))

GLPK Simplex Optimizer, v4.52
3 rows, 4 columns, 10 non-zeros
Objective scale factor = 200
      0: obj =   0.000000000e+00  infeas =  1.500e+01 (3)
*     3: obj =   5.035196687e+00  infeas =  0.000e+00 (0)
*     4: obj =   4.923076923e+00  infeas =  0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
  0.000517 seconds (915 allocations: 56.406 KiB)
JuMP.hasresultvalues(model) = true
JuMP.terminationstatus(model) == MOI.Success = true
JuMP.primalstatus(model) == MOI.FeasiblePoint = true
Objective value: 4.923076923076923
Optimal solution is x = 
[0.423077, 0.346154, 0.692308, 0.0]


**Note:** 
```julia
with_optimizer(GLPK.GLPKOptimizerLP, msg_lev = 4)
```
builds a factory that can create instances of 
```julia
GLPK.GLPKOptimizerLP(msg_lev = 4)
```
to re-build the internal model (or copy it).

### The Other Extreme: Direct Mode

Direct connection to a solver. All data is stored with solver, MOI does not save or cache anything.
* Pros: Access to complete solver interfase (e.g. callbacks). Allowed modifications are as efficient as possible for the solver. 
* Cons: Cannot apply incremental modifications not supported by the solver. Cannot copy the mode. Cannot use [Constraint Bridges](#Constraint-Bridges)

In [96]:
model = JuMP.direct_model(GLPK.GLPKOptimizerLP(msg_lev = 4))
@variable(model, x[1:n] >= 0) # Models x >=0
@constraint(model, [i=1:m], sum(A[i,j]*x[j] for j in 1:n) == b[i])
@objective(model, Min, sum(c[j]*x[j] for j in 1:n)) 

#SOLVE IT
#--------

# finally optimize the model
@time begin
JuMP.optimize(model) # solves the model
end

# TEST SOLVER STATUSES and DISPLAY THE RESULTS
#---------------------------------------------
@show JuMP.hasresultvalues(model)
@show JuMP.terminationstatus(model) == MOI.Success
@show JuMP.primalstatus(model) == MOI.FeasiblePoint

println("Objective value: ", JuMP.objectivevalue(model))
println("Optimal solution is x = \n", JuMP.resultvalue.(x))

GLPK Simplex Optimizer, v4.52
3 rows, 4 columns, 10 non-zeros
Objective scale factor = 200
      0: obj =   0.000000000e+00  infeas =  1.500e+01 (3)
*     3: obj =   5.035196687e+00  infeas =  0.000e+00 (0)
*     4: obj =   4.923076923e+00  infeas =  0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
  0.000099 seconds (39 allocations: 1.016 KiB)
JuMP.hasresultvalues(model) = true
JuMP.terminationstatus(model) == MOI.Success = true
JuMP.primalstatus(model) == MOI.FeasiblePoint = true
Objective value: 4.923076923076923
Optimal solution is x = 
[0.423077, 0.346154, 0.692308, 0.0]


In [97]:
status

### Intermediate Mode: Manual Mode = Direct Control of `CachingOptimizer`

In [24]:
model = JuMP.Model(with_optimizer(GLPK.GLPKOptimizerLP, msg_lev = 4),caching_mode = MOIU.Manual)

@variable(model, x[1:n] >= 0) 
@constraint(model, [i=1:m], sum(A[i,j]*x[j] for j in 1:n) == b[i])
@objective(model, Min, sum(c[j]*x[j] for j in 1:n)) 

In [25]:
JuMP.optimize(model)

LoadError: [91mAssertionError: m.state == AttachedOptimizer[39m

Need to control `CachingOptimizer` directly:

In [26]:
copt = JuMP.caching_optimizer(model)
typeof(copt)

MathOptInterface.Utilities.CachingOptimizer{MathOptInterface.AbstractOptimizer,MathOptInterface.Utilities.UniversalFallback{JuMP.JuMPMOIModel{Float64}}}

In [27]:
MOIU.CachingOptimizerState

Enum MathOptInterface.Utilities.CachingOptimizerState:
NoOptimizer = 0
EmptyOptimizer = 1
AttachedOptimizer = 2

In [28]:
MOIU.state(copt) == MOIU.EmptyOptimizer

true

`EmptyOptimizer` has not loaded the model. We can load it with `attachoptimizer!` and then we can solve:

In [29]:
MOIU.attachoptimizer!(copt);

In [30]:
MOIU.state(copt) == MOIU.AttachedOptimizer

true

In [31]:
JuMP.optimize(model)

GLPK Simplex Optimizer, v4.52
3 rows, 4 columns, 10 non-zeros
Objective scale factor = 200
      0: obj =   0.000000000e+00  infeas =  1.500e+01 (3)
*     3: obj =   5.035196687e+00  infeas =  0.000e+00 (0)
*     4: obj =   4.923076923e+00  infeas =  0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND


Unsuported incremental modifications through an error for `AttachedOptimizer`, but not for `EmptyOptimizer`.
We can return to `EmptyOptimizer` with `resetoptimizer`:

In [32]:
MOIU.resetoptimizer!(copt);
MOIU.state(copt) == MOIU.EmptyOptimizer

true

### Optional Cbc example

In [36]:
using Cbc
model = JuMP.Model(with_optimizer(Cbc.CbcOptimizer),caching_mode = MOIU.Manual)

@variable(model, x[1:n] >= 0) 
@constraint(model, [i=1:m], sum(A[i,j]*x[j] for j in 1:n) == b[i])
@objective(model, Min, sum(c[j]*x[j] for j in 1:n)) 

In [37]:
copt = JuMP.caching_optimizer(model)
MOIU.attachoptimizer!(copt);
JuMP.optimize(model)
@show JuMP.terminationstatus(model) == MOI.Success

true

JuMP.terminationstatus(model) == MOI.Success = true
Welcome to the CBC MILP Solver 
Version: 2.9.7 
Build Date: Oct 17 2016 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Presolve 3 (0) rows, 4 (0) columns and 10 (0) elements
0  Obj 0 Primal inf 4.0685937 (3)
4  Obj 4.9230769
Optimal - objective value 4.9230769
Optimal objective 4.923076923 - 4 iterations time 0.002
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



In [38]:
@constraint(model, x[1]==1)

LoadError: [91mAssertionError: MOI.canaddconstraint(m, typeof(func), typeof(set))[39m

In [39]:
MOIU.resetoptimizer!(copt);
@constraint(model, x[1]==1)
MOIU.attachoptimizer!(copt);
JuMP.optimize(model)

Welcome to the CBC MILP Solver 
Version: 2.9.7 
Build Date: Oct 17 2016 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Presolve determined that the problem was infeasible with tolerance of 1e-08
Analysis indicates model infeasible or unbounded
0  Obj 0 Primal inf 4.6459439 (4)
4  Obj 8 Primal inf 1.2936812 (1)
4  Obj 8 Primal inf 1.2936812 (1)
Primal infeasible - objective value 8
PrimalInfeasible objective 8 - 4 iterations time 0.002

Result - Linear relaxation infeasible

Enumerated nodes:           0
Total iterations:           0
Time (CPU seconds):         0.00
Time (Wallclock Seconds):   0.00

Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



## Constraint Bridges

There are multiple ways to write a constraint.

### Example 1:

In [62]:
model = JuMP.Model()
@variable(model,x[1:10])

10-element Array{JuMP.VariableRef,1}:
 x[1] 
 x[2] 
 x[3] 
 x[4] 
 x[5] 
 x[6] 
 x[7] 
 x[8] 
 x[9] 
 x[10]

Two-sided linear constraint as one `ScalarAffineFunction`-in-`Interval`:

In [48]:
@constraint(model, interval_inequality, -2 <= sum(x) <= 1)

interval_inequality : x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] ∈ [-2.0, 1.0]

In [49]:
interval_inequality.index

MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.Interval{Float64}}(2)

Two-sided linear constraint as two `ScalarAffineFunction`-in-`LessThan`:

In [63]:
@constraint(model, lower_bound, -sum(x) <= 2)

lower_bound : -x[1] - x[2] - x[3] - x[4] - x[5] - x[6] - x[7] - x[8] - x[9] - x[10] ≤ 2.0

In [64]:
lower_bound.index

MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}}(1)

In [57]:
@constraint(model, upper_bound,       sum(x) <= 1)

upper_bound : x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] ≤ 1.0

In [58]:
upper_bound.index

MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}}(3)

### Example 2:

In [67]:
model = JuMP.Model()
@variable(model,x)
@variable(model,y)
@variable(model,t)

t

Quadratic inequality as `ScalarQuadraticFunction`-in-`LessThan`:

In [70]:
@constraint(model,quadratic, x^2 + y^2 <= t)

quadratic : x² + y² - t ≤ 0.0

In [71]:
quadratic.index

MathOptInterface.ConstraintIndex{MathOptInterface.ScalarQuadraticFunction{Float64},MathOptInterface.LessThan{Float64}}(3)

Quadratic inequality as `VectorAffineFunction`-in-`SecondOrderCone`:

In [72]:
@constraint(model, conic, [(1+t)/2; (1-t)/2; x; y] in SecondOrderCone())

conic : [0.5 t + 0.5, -0.5 t + 0.5, x, y] ∈ MathOptInterface.SecondOrderCone(4)

In [73]:
conic.index

MathOptInterface.ConstraintIndex{MathOptInterface.VectorAffineFunction{Float64},MathOptInterface.SecondOrderCone}(4)

[Constraint Bridges](http://www.juliaopt.org/MathOptInterface.jl/stable/apimanual.html#Constraint-bridges-1) provide automatic transformation rules.

For instance, [Automatic Mode](#One-Extreme:-Automatic-Mode-=-"Almost"-Old-JuMP) actually adds two intermadiate layers: 
1. `CachingOptimizer` for incremental modifications, and
2. `LazyBridgeOptimizer <: AbstractBridgeOptimizer` for default automatic transformations

`LazyBridgeOptimizer` can be disabled with the `bridge_constraints = false` keyword argument.

### Example for `bridge_constraints`

In [78]:
using ECOS
model = JuMP.Model(with_optimizer(ECOS.ECOSOptimizer), bridge_constraints = false)
@variable(model,x)
@variable(model,y)
@variable(model,t)
@constraint(model,quadratic, x^2 + y^2 <= t)

LoadError: [91mConstraints of type MathOptInterface.ScalarQuadraticFunction{Float64}-in-MathOptInterface.LessThan{Float64} are not supported by the solver, try using `bridge_constraints=true` in the `JuMP.Model` constructor if you believe the constraint can be reformulated to constraints supported by the solver.[39m

In [80]:
model = JuMP.Model(with_optimizer(ECOS.ECOSOptimizer)) #, bridge_constraints = true)
@variable(model,x)
@variable(model,y)
@variable(model,t)
@constraint(model,quadratic, x^2 + y^2 <= t)

LoadError: [91mConstraints of type MathOptInterface.ScalarQuadraticFunction{Float64}-in-MathOptInterface.LessThan{Float64} are not supported by the solver and there are no bridges that can reformulate it into supported constraints.[39m

Missing bridges https://github.com/JuliaOpt/MathOptInterface.jl/issues/423

unsupported constraints https://github.com/JuliaOpt/JuMP.jl/pull/1386

In [71]:
@constraintref [1:3]

LoadError: [91mUndefVarError: @constraintref not defined[39m

In [72]:
@constraintref constraintName[1:3]

LoadError: [91mUndefVarError: @constraintref not defined[39m